In [204]:
import time
import math
import re
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import undetected_chromedriver as uc

In [337]:
# Fonction pour extraire les nombres et calculer les pages
def extraire_infos(texte):
    """
    Extrait le nombre de commentaires par page, le nombre total de commentaires,
    et calcule le nombre de pages à partir d'un texte donné.
    """
    # Nettoyer le texte pour supprimer les espaces insécables
    texte = texte.replace("\u202f", "")  # Remplace les espaces insécables par rien

    # Extraire les chiffres du texte
    chiffres = [int(s) for s in re.findall(r'\d+', texte)]
    
    if len(chiffres) >= 2:
        nb_commentaires_par_page = chiffres[1]  # Exemple : "15" (2e chiffre)
        nb_total_commentaires = chiffres[-1]   # Exemple : "1300" (dernier chiffre)
        nb_pages = math.ceil(nb_total_commentaires / nb_commentaires_par_page)
        return nb_commentaires_par_page, nb_total_commentaires, nb_pages
    else:
        return None, None, None


def scraper_infos_restaurant(driver):
    """
    Scrape les informations globales sur le restaurant.
    """
    nom = driver.find_element(By.XPATH, "//h1[@class='biGQs _P egaXP rRtyp']").text  
    adresse = driver.find_element(By.XPATH, "//div[contains(text(), 'Emplacement et coordonnées')]/following::span[contains(@class, 'biGQs _P pZUbB hmDzD')][1]").text 
    classement_element = driver.find_element(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB hmDzD')]//b/span").text.strip()
    classement = (re.search(r'\d+', classement_element).group())
    fourchette_prix = driver.find_element(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD') and contains(text(), '€')]").text.strip().replace("€", "").replace("\xa0", "")
    cuisine_element = driver.find_element(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD') and not(contains(text(), '€'))]")
    type_cuisine = [c.strip() for c in cuisine_element.text.split(',')]
    return {"nom": nom, "adresse": adresse, "classement": classement, "fourchette_prix": fourchette_prix, "type_cuisine": type_cuisine}

# Fonction pour scraper les avis d'une page
def scraper_page(driver):
    """
    Récupère les avis d'une seule page.
    """
    data = []
    # Récupération des éléments sur la page
    pseudos = driver.find_elements(By.XPATH, "//span[@class='biGQs _P fiohW fOtGX']")
    titres = driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']")
    etoiles = driver.find_elements(By.XPATH, "//div[@class='OSBmi J k']")
    nb_etoiles = [re.search(r'(\d+),', etoile.get_attribute("textContent")).group(1) for etoile in etoiles]
    dates = [driver.execute_script("return arguments[0].childNodes[0].textContent;", elem).strip() for elem in driver.find_elements(By.XPATH, "//div[@class='aVuQn']")]
    experiences = driver.find_elements(By.XPATH, "//span[@class='DlAxN']")
    reviews = driver.find_elements(By.XPATH, "//div[@data-test-target='review-body']//span[@class='JguWG' and not(ancestor::div[contains(@class, 'csNQI')])]")

    for i in range(len(titres)):
        avis = {
            "pseudo": pseudos[i].text if i < len(pseudos) else "",
            "titre_review": titres[i].text if i < len(titres) else "",
            "nb_etoiles": nb_etoiles[i] if i < len(nb_etoiles) else "",
            "date": dates[i] if i < len(dates) else "",
            "experience": experiences[i].text if i < len(experiences) else "",
            "review": reviews[i].text if i < len(reviews) else ""
        }
        data.append(avis)
    return data

# Fonction pour scraper les avis de toutes les pages
def scraper_toutes_pages(driver, nb_pages):
    """
    Scrape les avis de toutes les pages en utilisant la fonction `scraper_page`.
    """
    all_data = []
    actions = ActionChains(driver)
     
    for page in range(1, nb_pages + 1):
        print(f"Scraping de la page {page}...")
        time.sleep(5) 
        try:
            # Recharger les avis dynamiquement pour chaque page
            data = scraper_page(driver)
            print(f"Données collectées pour la page {page} : {len(data)} avis")
            all_data.extend(data)

            # Navigation vers la page suivante
            next_button = WebDriverWait(driver, 50).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Page suivante']"))
            )

            # Scroll et clic
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(5)
            actions.move_to_element(next_button).click().perform()


            print("Page suivante chargée.")
        
        except Exception as e:
            print(f"Erreur rencontrée à la page {page} : {e}")
            break  # Arrêter la boucle, mais conserver les données collectées jusqu'ici

    return all_data

def test_scraping(driver, nbPages_texte):
    """
    Teste l'ensemble du processus de scraping :
    - Extraction d'informations globales sur le restaurant
    - Scraping des avis sur toutes les pages
    - Regroupement des données
    """
    avis = []  # Initialiser pour éviter les erreurs
    infos_restaurant = {
        "nom": "Non disponible",
        "adresse": "Non disponible",
        "classement": "Non disponible",
        "fourchette_prix": "Non disponible",
        "type_cuisine": []
    }

    try:
        # Étape 1 : Extraire les infos globales
        infos_restaurant = scraper_infos_restaurant(driver)
        print(f"Nom : {infos_restaurant['nom']}")
        print(f"Adresse : {infos_restaurant['adresse']}")
        print(f"Classement : {infos_restaurant['classement']}")
        print(f"Fourchette de prix : {infos_restaurant['fourchette_prix']}")
        print(f"Type de cuisine : {infos_restaurant['type_cuisine']}")

        # Étape 2 : Extraire les infos pour les pages d'avis
        nb_commentaires_par_page, nb_total_commentaires, nb_pages = extraire_infos(nbPages_texte)
        print(f"Nombre de pages : {nb_pages}")

        # **Estimation du temps total** :
        average_time_per_page = 15  # Temps moyen par page en secondes
        estimated_total_time = average_time_per_page * nb_pages
        # Arrondir en minutes
        estimated_total_time_minutes = math.ceil(estimated_total_time / 60)
        print(f"Temps estimé pour terminer le scraping : {estimated_total_time_minutes} minutes.\n")

        # Étape 3 : Scraper les avis
        avis = scraper_toutes_pages(driver, nb_pages)
        print(f"Scraping terminé. Total d'avis collectés : {len(avis)}")

    except Exception as e:
        print(f"Erreur générale : {e}")

    # Étape 4 : Regrouper les données, même partielles
    restaurant_data = {
        "nom": infos_restaurant["nom"],
        "adresse": infos_restaurant["adresse"],
        "classement": infos_restaurant["classement"],
        "fourchette_prix": infos_restaurant["fourchette_prix"],
        "type_cuisine": infos_restaurant["type_cuisine"],
        "avis": avis  # Liste des avis
    }

    return restaurant_data

In [291]:
# Service pour ChromeDriver
# Modifier avec le bon chemin
service = Service('C:/Users/Ihnhn/Desktop/M2 SISE/NLP/Projet/chromedriver.exe')
# Step 3: Rotate user agents 
user_agents = [
    # Add your list of user agents here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
  
]
# Configuration du navigateur
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")

# select random user agent
user_agent = random.choice(user_agents)
# pass in selected user agent as an argument
options.add_argument(f'--user-agent={user_agent}')
# Lancement du navigateur
driver = uc.Chrome(options=options, service=service)

# Ouvrez TripAdvisor
#driver.get("https://www.tripadvisor.fr/Restaurant_Review-g187265-d3727154-Reviews-Les_Terrasses_de_Lyon-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
#driver.get("https://www.tripadvisor.fr/Restaurant_Review-g187265-d23110895-Reviews-Frazarin-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
driver.get("https://www.tripadvisor.fr/Restaurant_Review-g187265-d2281210-Reviews-Bouchon_Les_Lyonnais-Lyon_Rhone_Auvergne_Rhone_Alpes.html")
# Exécution de JavaScript pour rendre Selenium indétectable
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
time.sleep(3)
click_cookies = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[id='onetrust-reject-all-handler']"))).click()

print("Page ouverte avec un User-Agent réaliste")

Page ouverte avec un User-Agent réaliste


In [292]:
user_agent

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [327]:
nbPages_texte = driver.find_element("xpath", "//div[@class='Ci']").text

In [338]:
Bouchon_Les_Lyonnais = test_scraping(driver, nbPages_texte)

Nom : Bouchon Les Lyonnais
Adresse : 19 Rue de la Bombarde Angle Rue Tramassac, 69005 Lyon France
Classement : 187
Fourchette de prix : Italienne • -
Type de cuisine : ['Française', 'Européenne']
Nombre de pages : 87
Temps estimé pour terminer le scraping : 22 minutes.

Scraping de la page 1...


KeyboardInterrupt: 